In [23]:
import numpy as np
from docplex.mp.model import Model
# from docplex.mp.callbacks.cb_mixin import ConstraintCallbackMixin
# from docplex.mp.callbacks.cb_mixin import LazyConstraintCallback
import matplotlib.pyplot as plt

In [24]:
# 需要点と施設候補の座標データ
demand_points = [(2, 22), (42, 6), (48, 50), (32, 40), (16, 10),
                 (10, 34), (8, 30), (29, 16), (44, 34), (3, 16)]
candidate_sites = [(36, 32), (38, 32), (44, 11), (26, 50), (22, 28),
                   (32, 46), (20, 37), (35, 26), (2, 4), (18, 1)]


In [25]:
J = len(candidate_sites)  # 施設候補の数
D = len(demand_points)  # 需要点の数

In [26]:
# ユークリッド距離の計算
def compute_distances(demand_points, candidate_sites):
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d][j] = np.sqrt((demand_points[d][0] - candidate_sites[j][0])**2 +
                                      (demand_points[d][1] - candidate_sites[j][1])**2)
    return distances

In [27]:
distances = compute_distances(demand_points, candidate_sites)

In [ ]:
print

In [ ]:
# Lazy Constraint Callback 修正
class LazyCallback(LazyConstraintCallback):
    def __init__(self, model, x, theta):
        super().__init__(model)  # 修正：引数は model のみ
        self.x = x
        self.theta = theta
    
    def invoke(self, context):
        x_hat = {j: context.get_candidate_value(self.x[j]) for j in range(J)}
        theta_hat = context.get_candidate_value(self.theta)
        
        # ダミーの L(x, y) を計算（本来は separation problem を解く）
        y_hat = np.random.choice([0, 1], size=D)
        L_x_y = np.sum(y_hat)
        
        if theta_hat > L_x_y:
            context.add_lazy_constraint(self.theta <= L_x_y)  # 修正：lazy constraint の追加

NameError: name 'LazyConstraintCallback' is not defined

In [ ]:
# CPLEX モデルの作成と実行
def solve_s_cflp():
    mdl = Model("S-CFLP")
    x = mdl.binary_var_dict(range(J), name="x")
    theta = mdl.continuous_var(name="theta")
    
    mdl.maximize(theta)
    
    mdl.add_constraint(mdl.sum(x[j] for j in range(J)) <= 5)  # 施設数の制限
    
    mdl.register_callback(LazyCallback(mdl, x, theta))
    mdl.solve()
    
    BestSol = {j: x[j].solution_value for j in range(J)}
    theta_LB = theta.solution_value
    
    return BestSol, theta_LB

In [ ]:
# 最適解の計算とプロット
best_solution, best_theta = solve_s_cflp()
print("Best Solution:", best_solution)
print("Best Theta:", best_theta)

# 結果の可視化
def plot_results():
    plt.figure(figsize=(8, 8))
    plt.scatter(*zip(*demand_points), c='blue', marker='o', label='Demand Points')
    plt.scatter(*zip(*candidate_sites), c='red', marker='s', label='Candidate Sites')
    
    for j, val in best_solution.items():
        if val > 0.5:
            plt.scatter(candidate_sites[j][0], candidate_sites[j][1], c='green', marker='s', s=100, label='Selected Facility' if j == 0 else "")
    
    plt.legend()
    plt.title("Optimal Facility Location")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.show()

plot_results()

TypeError: __init__() takes 1 positional argument but 2 were given